In [2]:
from langsmith import Client
from langsmith.utils import LangSmithNotFoundError

# Your questions and answers
# Define questions and answers for the golden dataset
question_answers = {
    # Basic Retrieval
    "What was the middle-period population count for Zlín Region?": "579562",


    # Comparisons
    "List regions where mid-period male population exceeded 300,000":
        "Central Bohemia, Prague, South Moravia, Moravian-Silesia, Pilsner, South Bohemia, Olomouc, Ústí",
    "Was there any quarter where Plzeň's women outnumbered men by >15,000?":
        "No",


    # Aggregations
    "What was the maximum female population recorded in any region?": "743608",
    "What was the average male population across all regions at period end?": "381994.5",
    "Count how many regions had <500k total residents mid-period": "2",


    # Temporal/Deltas
    "What was South Bohemia's population change rate per month?": "-128.77 per month",


    # Demographic Ratios
    "Which region had the most balanced gender ratio at period end?": "Vysočina",
    "What percentage of Pardubice region was female at mid-period?": "50.57%",


    # Complex Joins/Logic
    "List regions where women population decreased >1000":
        "Ústí (1466), Moravian-Silesia (3299), Karlovarský kraj (1228), Liberecký kraj (1142), Plzeňský kraj (1073)",


    # Edge Cases

}

ls_client = Client()

# Get or create dataset
try:
    dataset = ls_client.read_dataset(dataset_name="czsu agent problematic1")
    print(f"Dataset '{dataset.name}' found with ID: {dataset.id}")
except LangSmithNotFoundError:
    dataset = ls_client.create_dataset(
        dataset_name="czsu agent problematic1",
        description="Dataset of Czech Statistical Office agent questions and answers."
    )
    print(f"Dataset '{dataset.name}' created with ID: {dataset.id}")

# Fetch existing examples' questions to avoid duplicates
existing_examples = ls_client.list_examples(dataset_id=dataset.id)
existing_questions = set()
for ex in existing_examples:
    q = ex.inputs.get("question")
    if q:
        existing_questions.add(q)

# Prepare only new examples (filter out duplicates)
new_examples = []
for question, answer in question_answers.items():
    if question not in existing_questions:
        new_examples.append({"inputs": {"question": question}, "outputs": {"answers": answer}})

if new_examples:
    ls_client.create_examples(dataset_id=dataset.id, examples=new_examples)
    print(f"Added {len(new_examples)} new examples to dataset '{dataset.name}'.")
else:
    print("No new examples to add; all questions already exist in the dataset.")

Dataset 'czsu agent problematic1' found with ID: be16fded-6640-48e6-b2cd-2c87fdc7dfbb
Added 8 new examples to dataset 'czsu agent problematic1'.
